In [6]:
import warnings
warnings.filterwarnings("ignore")

from config import auth

In [83]:
from langchain_community.chat_models.gigachat import GigaChat

chat = GigaChat(
    credentials=auth,
    model="GigaChat:latest",
    verify_ssl_certs=False
)

In [84]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False
)

In [85]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('python_basics.pdf')
splitted_data = loader.load_and_split(text_splitter)

In [86]:
len(splitted_data)

97

https://github.com/avidale/encodechka

In [13]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model = "BAAI/bge-m3"
embedding = HuggingFaceEmbeddings(model_name=model)

In [87]:
from langchain_chroma import Chroma

vector_store = Chroma.from_documents(splitted_data, embedding=embedding, persist_directory="./chroma")
# vector_store = Chroma(persist_directory="./chroma", embedding_function=embedding)

In [111]:
vector_store.similarity_search("Специальные типы", k=3)[0].page_content

'Карта встроенных типов (с именами функций для приведения  \nк нужному типу и именами классов для наследования от этих типов): \n1. Специальн\nые тип\nы: None, NotIm\nplemented и Ell\nipsis; \n2. Чис\nла:'

In [91]:
embedding_retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [130]:
system_prompt = """
Ты работаешь как интеллектуальная система, которая создает тесты на основе предоставленных учебных материалов. Твои задачи:
1. Генерировать вопрос по содержанию предоставленных текстов.
2. Создать четыре варианта ответа на вопрос, где один из них правильный, а остальные три - неправильные, но правдоподобные.
3. Указывать правильный ответ для каждого вопроса.

### Пример:

**Текст:**
Python - это высокоуровневый язык программирования, который используется для различных типов программирования, включая веб-разработку, автоматизацию и анализ данных.

**Вопрос:**
Какой язык программирования используется для веб-разработки, автоматизации и анализа данных?

**Варианты ответа:**
A) Java
B) C++
C) Python
D) JavaScript

**Правильный ответ:**
C) Python

### Формат:

**Текст:**
{context}

**Вопрос:**
<сгенерированный вопрос>

**Варианты ответа:**
A) <вариант ответа 1>
B) <вариант ответа 2>
C) <вариант ответа 3>
D) <вариант ответа 4>

**Правильный ответ:**
<правильный вариант ответа>
"""

In [131]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(system_prompt)

In [139]:
prompt = ChatPromptTemplate.from_template('''Создай вопрос и 4 варианта ответов на основе запроса пользователя. \
1. Сформулируй вопрос, основанный на предоставленном контексте. \
2. Придумай 4 варианта ответа, один из которых правильный, а остальные три - неправильные, но правдоподобные. \
3. Укажи правильный ответ. \
Используй при этом только информацию из контекста. Если в контексте нет \
информации для ответа, сообщи об этом пользователю.
Контекст: {context}
Вопрос: {input}
Ответ:'''
)

In [140]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(
    llm=chat,
    prompt=prompt
)

In [141]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(embedding_retriever, document_chain)

In [143]:
retrieval_chain.invoke({'input': 'Напиши вопрос и варианты ответов по теме Типы данных'})

{'input': 'Напиши вопрос и варианты ответов по теме Типы данных',
 'context': [Document(metadata={'page': 9, 'source': 'python_basics.pdf'}, page_content='функции type(). \n \n2.1. Тип int и long \nДва типа: int (целые числа) и long (целые произвольной точн о-\nсти) – служат моделью для представления целых чисел. Первый соо т-'),
  Document(metadata={'page': 20, 'source': 'python_basics.pdf'}, page_content='ний. Для удобства функции можно условно разделить на следующие \nкатегории:  \n \nКатего\nрия Функ\nции \n1 2 \nФунк\nции преобразования  \nтипо\nв и классы  coerce, str, repr, int, \nlist,tup\nle,lon\ng,flo'),
  Document(metadata={'page': 23, 'source': 'python_basics.pdf'}, page_content='типов данных. В старых версиях Python для преобразования к ну ж-\nному типу использовалась одноименная функция. В новых версиях \nPython роль таких функций играют имена встроенных классов (одн а-')],
 'answer': 'Вопрос: Какие два типа данных служат моделью для представления целых чисел?\nВарианты о

In [117]:
system_prompt = """
Ты - помощник по созданию тестов. Твоя задача - генерировать вопросы теста на основе учебных материалов. Для каждого вопроса ты должен:
1. Сформулировать вопрос, основанный на предоставленном тексте.
2. Придумать 4 варианта ответа, один из которых правильный, а остальные три - неправильные, но правдоподобные.
3. Указать правильный ответ.
"""

In [118]:
content = '''
Встроенные типы данных в Python
Питон работает с двумя категориями данных – встроенными типами (они поддерживаются по умолчанию) и специализированными (для операций с ними нужно подключение определенного модуля). К специализированным типам данных относятся, например, datetime (дата и время) и deque (двухсторонняя очередь).

Все встроенные типы данных в Python можно разделить на следующие группы:

Числовые – целые, вещественные, комплексные числа. Примечание: для максимально точных расчетов с десятичными числами в Python используют модуль decimal (тип данных Decimal), а для операций с рациональными числами (дробями) – модуль fractions (тип данных Fraction).
Булевы – логические значения True (истина) и False (ложь).
Строковые – последовательности символов в кодировке Unicode.
NoneType – нейтральное пустое значение, аналогичное null в других языках программирования.
Последовательности – списки, кортежи, диапазоны.
Словари – структура данных типа «ключ: значение».
Множества – контейнеры, содержащие уникальные значения. Подразделяются на изменяемые set и неизменяемые frozenset множества.
Байтовые типы – bytes (байты), bytearray(изменяемая байтовая строка), memoryview(предоставление доступа к внутренним данным объекта).
'''

In [119]:
from langchain.schema import HumanMessage, SystemMessage

msgs = [
    SystemMessage(content=content)
]

answer = chat(messages=msgs, prompt=system_prompt)
answer.content

'Встроенные типы данных в Python включают:\n\n1. Числовые типы:\n   - int (целые числа)\n   - float (вещественные числа)\n   - complex (комплексные числа)\n\n2. Булевы типы:\n   - bool (логические значения True или False)\n\n3. Строковые типы:\n   - str (строки символов в кодировке Unicode)\n\n4. Тип NoneType:\n   - None (нейтральное пустое значение)\n\n5. Последовательности:\n   - list (списки)\n   - tuple (кортежи)\n   - range (диапазоны)\n\n6. Словари:\n   - dict (структура данных "ключ: значение")\n\n7. Множества:\n   - set (изменяемое множество)\n   - frozenset (неизменяемое множество)\n\n8. Байтовые типы:\n   - bytes (байты)\n   - bytearray (изменяемая байтовая строка)\n   - memoryview (предоставление доступа к внутренним данным объекта)'